# Simulation Module

## Import Library

In [1]:
! pip install simpy

In [2]:
import simpy
import statistics

## Wrapper Clases

In [3]:
# Generic helper class to hold information regarding to resources
# This simplifies how we pass information from main program to entity process
class Moderator(object):
    def __init__(self, env, name, market, productivity, utilisation, handling_time, accuracy):
        self.env = env
        self.name = "Moderator " + name # important
        self.market = market # important
        self.productivity = productivity
        self.utilisation = utilisation
        self.handling_time = handling_time
        self.accuracy = accuracy
        self.resource = simpy.Resource(env) # serve 1, unlimited in queue
        self.task = 0

    def print_stats(self):
        print("\t[{}] {} moderating, {} in queue".format(self.name, self.resource.count, len(self.resource.queue)))

    def get_handling_time(self):
        return self.handling_time

    def get_queue_size(self):
        return self.resource.count + len(self.resource.queue)
    
    def get_utilisation(self):
        return self.task/self.productivity

# Generic helper class to hold information regarding to processes
class Advertisement(object):
    def __init__(self, name, market, date, score, profit):
        self.name = "Advertisement " + name
        self.market = market
        self.date = date
        self.score = score
        self.profit = profit

## Function

In [4]:
# Input: advertisements
# Output: 2 arrays of advertisement and moderators
# mismatch score, utilisation score, profit
def loss(env, advertisements, moderators):
    abs_mismatch_list = []
    # helper class for moderation

    def dispatcher(moderators): # looting
        assigned_queue_size = -1
        assigned_moderator = None
        for moderator in moderators:
            queue_size = moderator.resource.count + len(moderator.resource.queue)
            if queue_size < assigned_queue_size or assigned_queue_size == -1:
                assigned_queue_size = queue_size
                assigned_moderator = moderator
        return assigned_moderator, assigned_queue_size


    # moderation - Entity Process
    # Describe how advertisement performs at each moderator
    def moderation(env, advertisement, moderators):
        # arrival statement
        print("[{:6.2f}:{}] - arrive at dispatcher".format(env.now, advertisement.name))

        # Debugging statement to print state of moderators
        for moderator in moderators:
            moderator.print_stats()

        # dispatcher logic: looting
        # advertisement will choose a moderator based on the dispatcher function
        moderator, num_in_moderator = dispatcher(moderators)
        abs_mismatch_list.append(abs(moderator.accuracy - advertisement.score))
        print("[{:6.2f}:{}] - chooses {} with {} ads in queue".format(env.now, advertisement.name, moderator.name, num_in_moderator))
        
        # process logic to handle the request for a spot at the assigned moderator
        with moderator.resource.request() as request:
            yield request
            print('[{:6.2f}:{}] - begin moderation'.format(env.now, advertisement.name))
            handling_time = moderator.get_handling_time()
            yield env.timeout(handling_time)
            print('[{:6.2f}:{}] - finish moderation'.format(env.now, advertisement.name))
        # print('[{:6.2f}:{}] - depart from queue'.format(env.now, name))
        moderator.task += 1

    # moderation event generator - Supporting Process
    def moderation_event_generator(env, advertisements, moderators):
        for index in range(len(advertisements)):
            adv = advertisements[index]
            mod = moderation(env, adv, moderators)
            env.process(mod)
            try:
                next_adv = advertisements[index + 1]
                next_entity_arrival = next_adv.date - adv.date
            except:
                next_entity_arrival = 0
            yield env.timeout(next_entity_arrival)
            
    env.process(moderation_event_generator(env, advertisements, moderators))
    env.run()
    mismatch_score = statistics.mean(abs_mismatch_list)
    utilisation_score = statistics.mean([x.get_utilisation() for x in moderators])
    return mismatch_score, utilisation_score

## Test run

In [5]:
env = simpy.Environment()

# Advertisement List (name, market, date, score, profit)
advertisement1 = Advertisement("123", [1,2], 0.9, 0.8, 1000)
advertisement2 = Advertisement("121", [1,2], 0.9, 0.7, 1000)
advertisements = [advertisement1, advertisement2]
advertisements = sorted(advertisements, key = lambda x:x.date)

# Moderator List (env, name, market, productivity, utilisation, handling_time, accuracy)
moderator1 = Moderator(env, 'adam', [1,2], 5.1, 0.8, 10, 0.9)
moderator2 = Moderator(env, 'bell', [1,2], 2.1, 0.8, 10, 0.9)
moderators = [moderator1, moderator2]

loss(env, advertisements, moderators)

[  0.00:Advertisement 123] - arrive at dispatcher
	[Moderator adam] 0 moderating, 0 in queue
	[Moderator bell] 0 moderating, 0 in queue
[  0.00:Advertisement 123] - chooses Moderator adam with 0 ads in queue
[  0.00:Advertisement 121] - arrive at dispatcher
	[Moderator adam] 1 moderating, 0 in queue
	[Moderator bell] 0 moderating, 0 in queue
[  0.00:Advertisement 121] - chooses Moderator bell with 0 ads in queue
[  0.00:Advertisement 123] - begin moderation
[  0.00:Advertisement 121] - begin moderation
[ 10.00:Advertisement 123] - finish moderation
[ 10.00:Advertisement 121] - finish moderation


(0.15000000000000002, 0.33613445378151263)